### Custom Model
**Linear Model custom model as 1st without using 2nd library**

Lets create a custom model (equivalent model to LinearRegressor using the lower level TensorFlow API) that is not built into `TensorFlow`. We can still retain the high level abstraction of **data set, feeding, training, etc. of tf.contrib.learn**.

To define a custom model that works with tf.contrib.learn, we need to use **tf.contrib.learn.Estimator.** *tf.contrib.learn.LinearRegressor* is a sub-class of tf.contrib.learn.Estimator. Instead of sub-classing Estimator, we simply provide Estimator a function **model_fn** that tells tf.contrib.learn how it can evaluate predictions, training steps, and loss.

In [2]:
import time
start_time = time.time()
import numpy as np
import tensorflow as tf

# Eventually following is a function to built a custom model :----------
# Declare list of features, we only have one real-valued feature
def model(features, labels, mode):
  # Build a linear model and predict values
  W = tf.get_variable("W", [1], dtype=tf.float64)
  b = tf.get_variable("b", [1], dtype=tf.float64)
  y = W*features['x'] + b
  # Loss sub-graph
  loss = tf.reduce_sum(tf.square(y - labels))
  # Training sub-graph
  global_step = tf.train.get_global_step()
  optimizer = tf.train.GradientDescentOptimizer(0.01)
  train = tf.group(optimizer.minimize(loss),
                   tf.assign_add(global_step, 1))
  # ModelFnOps connects subgraphs we built to the
  # appropriate functionality.
  return tf.contrib.learn.ModelFnOps(
      mode=mode, predictions=y,
      loss=loss,
      train_op=train)

estimator = tf.contrib.learn.Estimator(model_fn=model)

# define our data sets
x_train = np.array([1., 2., 3., 4.])
y_train = np.array([0., -1., -2., -3.])
x_eval = np.array([2., 5., 8., 1.])
y_eval = np.array([-1.01, -4.1, -7, 0.])
input_data = tf.contrib.learn.io.numpy_input_fn({"x": x_train}, y_train, 4, num_epochs=1000)
eval_input_data = tf.contrib.learn.io.numpy_input_fn({"x":x_eval},y_eval, batch_size=4,num_epochs=1000)

# train
estimator.fit(input_fn=input_data, steps=1000)

# Here we evaluate how well our model did. 
train_loss = estimator.evaluate(input_fn=input_data)
eval_loss = estimator.evaluate(input_fn=eval_input_data)
print("train loss: %r"% train_loss)
print("eval loss: %r"% eval_loss)

# Time taken to complete the process
print(" The total time required to train and evaluate this model is --- %s seconds---" % (time.time()-start_time))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x00000256DB89DDD8>, '_keep_checkpoint_max': 5, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_save_checkpoints_steps': None, '_master': '', '_task_type': None, '_task_id': 0, '_save_checkpoints_secs': 600, '_session_config': None, '_is_chief': True, '_keep_checkpoint_every_n_hours': 10000, '_evaluation_master': '', '_tf_random_seed': None, '_environment': 'local', '_num_worker_replicas': 0, '_num_ps_replicas': 0, '_model_dir': 'C:\\Users\\Ripu\\AppData\\Local\\Temp\\tmped4z6r7e', '_save_summary_steps': 100}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into C:\Users\Ripu\AppData\Local\Temp\tmped4z6r7e\model.ckpt.
INFO:tensorflow:loss = 7.64801287674, step = 1
INFO:tensorflow:global_step/sec: 1075.84
INFO:tensorflow:loss = 0.000172553749623, step = 101 (0.095 sec)
INFO:tensorflow

#### OUTPUT
***train loss: {'loss': 5.7751757e-14, 'global_step': 1000}
eval loss: {'loss': 0.010100062, 'global_step': 1000}
The total time required to train and evaluate this model is --- 5.568190336227417 seconds---***